In [1]:
from rank_bm25 import BM25Okapi
import pandas as pd 
import re
from tqdm import tqdm
import numpy as np
import json
tqdm.pandas()

In [9]:
devices = []
rooms = []
for key, value in corpus_slot.items():
    for key_slot, value_slot in value.items():
        if 'device' in key_slot:
            devices.extend(value_slot['words']) 
        elif 'room' in key_slot:
            rooms.extend(value_slot['words'])

In [10]:
devices = set(devices)
print(devices)

{'loa thứ 4', 'quạt treo tường 3', 'hắt tường thứ 1', 'đèn treo tường thứ 1', 'cột đèn 1', 'đèn sắc màu', 'đèn compact 1', 'bình nóng lạnh 1', 'đèn ốp tường 4', 'van nước thứ 2', 'bóng sân 2', 'mành đôi thứ 2', 'âm thanh đa vùng thứ 3', 'đèn chùm thứ 3', 'đèn cảnh thứ 3', 'đèn thứ 3', 'huỳnh quang 1', 'đèn tường 3', 'bóng thờ 1', 'bóng kiểng 2', 'bóng làm việc 1', 'cửa cuốn thứ 1', 'bóng tiết kiệm điện thứ 1', 'đèn sưởi 1', 'đèn ngủ thứ 1', 'quạt thứ 1', 'đèn tiết kiệm điện', 'đèn treo tường thứ 3', 'quạt hút mùi thứ 4', 'rèm cửa 1', 'điều hòa thứ 1', 'huỳnh quang 2', 'loa còi thứ 2', 'bóng trụ cổng', 'đèn sân 4', 'vô tuyến thứ 3', 'đèn thả 1', 'đèn chùm thứ 1', 'bóng làm việc thứ 1', 'bóng hắt tường 2', 'bóng tròn', 'bóng ốp tường thứ 3', 'đèn âm trần 1', 'ốp tường 1', 'đèn trụ cổng thứ 3', 'đèn 1', 'động cơ thứ 2', 'đèn nhiều màu', 'đèn tranh 1', 'đèn tròn 3', 'đèn thứ 4', 'đèn led thứ 1', 'rèm ngang 1', 'bóng downlight thứ 1', 'mành thứ 3', 'mành cuốn 1', 'quạt thông gió thứ 4', 'rè

In [14]:
'chùm thứ 3' in devices

False

In [11]:
rooms = set(rooms)

In [6]:
data = pd.read_csv('/media/dmk/D:/Data/Project/NLP/NLUPractice/assets/data/bkai/train/train.csv')

In [7]:
def preprocess_text(sent):
    # sent = re.sub('[0-9]+', "", sent).strip()
    for device in devices:
        sent = re.sub(f" {device}", " ", sent)
    return re.sub('\s+', " ", sent)

In [8]:
data['text_copy'] = data['text'].astype(str).progress_apply(lambda x: preprocess_text(x))

100%|██████████| 1790/1790 [01:09<00:00, 25.79it/s]


In [12]:
all_sentences = data['text_copy'].values.tolist()
all_labels = data['intent'].values.tolist()

In [13]:
data

,text,intent,tag,text_copy
0,tăng bóng 3 26 phần trăm,smart.home.increase.percentage,O B-devicedevice I-devicedevice B-change-value...,tăng 26 phần trăm
1,hãy tăng thêm độ sáng phòng ngủ con trai lên 9...,smart.home.increase.percentage,O B-commandcommand I-commandcommand O O B-room...,hãy tăng thêm độ sáng phòng ngủ con trai lên 9...
2,giúp mình tăng đèn âm trần thứ 2 lên 10 phần t...,smart.home.increase.percentage,O O O B-devicedevice I-devicedevice I-devicede...,giúp mình tăng lên 10 phần trăm ở phòng con nh...
3,bạn có thể tăng giúp mình bóng chùm thứ 3 lên ...,smart.home.increase.percentage,O O O O O O B-devicedevice I-devicedevice I-de...,bạn có thể tăng giúp mình chùm thứ 3 lên mức 2...
4,tăng bình nóng lạnh 5 phần trăm,smart.home.increase.percentage,O B-devicedevice I-devicedevice I-devicedevice...,tăng 5 phần trăm
...,...,...,...,...
1785,xin chào,greeting,O O,xin chào
1786,trợ lý có đó chứ mình cần bạn,greeting,O O O O O O O O,trợ lý có đó chứ mình cần bạn
1787,mình là chaien hãy giúp mình đánh bại nobita nào,greeting,O O O O O O O O O O,mình là chaien hãy giúp mình đánh bại nobita nào
1788,alo trợ lý ơi,greeting,O O O O,alo trợ lý ơi


In [17]:
corpus_word_tokenizer = [sent.split(" ") for sent in all_sentences]

In [18]:
type(all_labels)

list

In [25]:
def check(sent):
    # sent = preprocess_text(sent)
    idx = all_sentences.index(sent)
    if idx == 0:
        corpus_word_tokenizer_temp = corpus_word_tokenizer[idx + 1 :]
        labels_temp = all_labels[idx+1 :]
    elif idx < len(all_sentences) - 1:
        corpus_word_tokenizer_temp = corpus_word_tokenizer[: idx] + corpus_word_tokenizer[idx + 1 : ]
        labels_temp = all_labels[:idx] + all_labels[idx + 1:]
        
    else:
        corpus_word_tokenizer_temp = corpus_word_tokenizer[:idx]
        labels_temp = all_labels[:idx]
    
    bm25_model = BM25Okapi(corpus_word_tokenizer_temp)
    sent = sent.split(" ")
    scores = bm25_model.get_scores(sent)
    index_score = np.argmax(scores)
    if index_score > idx:
        print(all_sentences[index_score+1], '---', scores[index_score])
    else:
        print(all_sentences[index_score], '---', scores[index_score])
    return labels_temp[index_score]
    
        

In [26]:
check('tăng bình nóng lạnh 5 phần trăm')

mở bình nóng lạnh thứ 4 sáng 68 phần trăm --- 15.50787945360508


'smart.home.set.percentage'